<a href="https://colab.research.google.com/github/Sushila-Phogat/CE888/blob/master/Lab8/Flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import os
os.environ['KAGGLE_USERNAME'] = "sushilakumari" # username from the json file
os.environ['KAGGLE_KEY'] = "87aa4a19a3b29f38f457fd53233688e6" # key from the json file
!kaggle datasets download -d alxmamaev/flowers-recognition # api copied from kaggle

flowers-recognition.zip: Skipping, found more recently modified local copy (use --force to force download)


In [27]:
!git clone https://github.com/sagihaider/CE888_2020.git

fatal: destination path 'CE888_2020' already exists and is not an empty directory.


In [28]:
# Upload data
from zipfile import ZipFile
file_name = "/content/flowers-recognition.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [29]:
!pip install tensorflow==1.3.0
!pip install keras==2.0.7

In [0]:
import numpy as np
import os
import time
from CE888_2020.Lab_8.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [31]:
#%% Loading the training data
import os
import sys
import shutil
PATH = os.getcwd()
print(PATH)
# Define data path
data_path = PATH + '/flowers'
data_dir_list = os.listdir(data_path)
shutil.rmtree('/content/flowers/.ipynb_checkpoints')
print(data_dir_list)


/content


FileNotFoundError: ignored

In [0]:
img_data_list=[] # created an empty list for images to be loaded

for dataset in data_dir_list:
	img_list=os.listdir(data_path+'/'+ dataset)
	print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
	print(img_list)
	for img in img_list:
		img_path = data_path + '/'+ dataset + '/'+ img
		img = image.load_img(img_path, target_size=(224, 224))
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
#		x = x/255
		# print('Input image shape:', x.shape)
		img_data_list.append(x)

#%% 
img_data = np.array(img_data_list)
#img_data = img_data.astype('float32')
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)